# Purpose 
### Feature Selection by optimizing FSJaya Algorithm. 

The challenge is to tweak the present algorithm for feature selection purposes. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

## Step 1: Load Dataset

The Datasets selected for evaluation purposes are: 
- mandelon
- musk 

Both datasets are accesbile from [add the name of the website]

### Madelon Dataset

- Data type: non-sparse
- Number of features: 500
- Number of examples and check-sums:
      	     Pos_ex	Neg_ex	Tot_ex	Check_sum
             

     Train	 1000	 1000	 2000	488083511.00
     

     Valid	  300	  300	  600	146395833.00
     

     Test	  900	  900	 1800	439209553.00
     

     All  	 2200	 2200	 4400	1073688897.00
     

In [2]:
madelon = pd.read_csv('madelon/madelon_train.data', sep=' ', header = None)
madelon.info()
madelon.head()

madelon_test = pd.read_csv('madelon/madelon_test.data', sep=' ', header = None)
madelon_test.info()
#madelon_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 501 entries, 0 to 500
dtypes: float64(1), int64(500)
memory usage: 7.6 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Columns: 501 entries, 0 to 500
dtypes: float64(1), int64(500)
memory usage: 6.9 MB


## Musk Dataset

4. Relevant Information:
   This dataset describes a set of **92** molecules of which **47** are judged
   by human experts to be musks and the remaining 45 molecules are
   judged to be non-musks.  The goal is to learn to predict whether
   new molecules will be musks or non-musks.  However, the 166 features
   that describe these molecules depend upon the exact shape, or
   conformation, of the molecule.  Because bonds can rotate, a single
   molecule can adopt many different shapes.  To generate this data
   set, the low-energy conformations of the molecules were generated
   and then filtered to remove highly similar conformations. This left
   476 conformations.  Then, a feature vector was extracted that
   describes each conformation.

   This many-to-one relationship between feature vectors and molecules
   is called the **"multiple instance problem"**.  When learning a
   classifier for this data, the classifier should classify a molecule
   as "musk" if ANY of its conformations is classified as a musk.  A
   molecule should be classified as "non-musk" if NONE of its
   conformations is classified as a musk.

5. Number of Instances  **476**

6. Number of Attributes **168** plus the class.

7. For Each Attribute:
   
   Attribute:           Description:
   molecule_name:       Symbolic name of each molecule.  Musks have names such
                        as MUSK-188.  Non-musks have names such as
                        NON-MUSK-jp13.
   conformation_name:   Symbolic name of each conformation.  These
                        have the format MOL_ISO+CONF, where MOL is the
                        molecule number, ISO is the stereoisomer
                        number (usually 1), and CONF is the
                        conformation number. 
   f1 through f162:     These are "distance features" along rays (see
                        paper cited above).  The distances are
                        measured in hundredths of Angstroms.  The
                        distances may be negative or positive, since
                        they are actually measured relative to an
                        origin placed along each ray.  The origin was
                        defined by a "consensus musk" surface that is
                        no longer used.  Hence, any experiments with
                        the data should treat these feature values as
                        lying on an arbitrary continuous scale.  In
                        particular, the algorithm should not make any
                        use of the zero point or the sign of each
                        feature value. 
   f163:                This is the distance of the oxygen atom in the
                        molecule to a designated point in 3-space.
                        This is also called OXY-DIS.
   f164:                OXY-X: X-displacement from the designated
                        point.
   f165:                OXY-Y: Y-displacement from the designated
                        point.
   f166:                OXY-Z: Z-displacement from the designated
                        point. 
   class:               0 => non-musk, 1 => musk

   Please note that the molecule_name and conformation_name attributes
   should not be used to predict the class.

8. Missing Attribute Values: none.

9. Class Distribution: 
   Musks:     47
   Non-musks: 45

In [3]:
musk =  pd.read_csv('musk/clean2.data/clean2.data', header = None)
musk.info()
musk.head()

musk_test =  pd.read_csv('musk/clean1.data/clean1.data', header = None)
#musk_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Columns: 169 entries, 0 to 168
dtypes: float64(1), int64(166), object(2)
memory usage: 8.5+ MB


## Step 2: Prepare data for Classifiers. 

According to the shared paper [A jaya algorithm based wrapper method for optimal feature selection in supervised classification] data has to be trained using following classifiers: 

    - NB (Naive Bayes)
    - KNN (K Nearest Neighbor)
    - LDA 
    - RT (Regression Tree)

#### Step 2a : Separating features and labels as X, Y variables  (Training Dataset)

In [4]:
Musk_X = musk.drop(168, axis = 1)
Musk_X = Musk_X.drop(0, axis=1)
Musk_X = Musk_X.drop(1, axis=1)
Musk_X.columns = np.arange(len(Musk_X.columns))
Musk_Y = musk[168]


Madelon_X = madelon.drop(500, axis = 1)
Y_labels = np.hstack([np.ones(1000), np.zeros(1000)])
Madelon_Y = pd.Series(Y_labels.T)


#### Step 2b : Separating features and labels as X, Y variables  (Test Dataset)

In [5]:
Musk_x = musk_test.drop(168, axis = 1)
Musk_x = Musk_x.drop(0, axis=1)
Musk_x = Musk_x.drop(1, axis=1)
Musk_x.columns = np.arange(len(Musk_x.columns))
Musk_y = musk_test[168]


Madelon_x = madelon_test.drop(500, axis = 1)
y_labels = np.hstack([np.ones(900), np.zeros(900)])
Madelon_y = pd.Series(y_labels.T)
len(Musk_x)
len(Musk_y)

print("Madelon TRAINING \n dataset: # of features: " , Madelon_X.shape[1], 
      '\n Number of Measurements: ', Madelon_X.shape[0], 
      'Y_shape: ', Madelon_Y.shape)

print("Madelon TEST \n dataset: # of features: " , Madelon_x.shape[1], 
      '\n Number of Measurements: ', Madelon_x.shape[0], 
      'Y_shape: ', Madelon_y.shape)

print("Musk TRAINING \n dataset: # of features: " , Musk_X.shape[1], 
      '\n Number of Measurements: ', Musk_X.shape[0],
      'Y_shape: ', Musk_Y.shape)

print("Musk TEST \n dataset: # of features: " , Musk_x.shape[1], 
      '\n Number of Measurements: ', Musk_x.shape[0], 
      'Y_shape: ', Musk_y.shape)

Madelon TRAINING 
 dataset: # of features:  500 
 Number of Measurements:  2000 Y_shape:  (2000,)
Madelon TEST 
 dataset: # of features:  500 
 Number of Measurements:  1800 Y_shape:  (1800,)
Musk TRAINING 
 dataset: # of features:  166 
 Number of Measurements:  6598 Y_shape:  (6598,)
Musk TEST 
 dataset: # of features:  166 
 Number of Measurements:  476 Y_shape:  (476,)


### Step 3: Classification

#### Step 3a: Train Classifiers without Feature Selection

In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from pandas_ml import ConfusionMatrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor

def data_classification(train_x, train_y, test_x = None ,test_y = None):
    names = ["NB", "RT", "KNN", "LDA"]

    classifiers = [
        GaussianNB(),
        DecisionTreeRegressor(),
        KNeighborsClassifier(n_neighbors=10),
        LinearDiscriminantAnalysis()
    ]
    model_return = {}
    for name, clf in zip(names, classifiers):
        
        clf.fit(train_x, np.ravel(train_y))
        print(clf)
        
        # TRAIN
        #probs = clf.predict_proba(train_x)
        pred_ytrain = clf.predict(train_x)
        score_ytrain = clf.score(train_x,np.ravel(train_y))
        
        #TEST
        #probs_t = clf.predict_proba(test_x)
        pred_ytest = clf.predict(test_x)
        score_ytest = clf.score(test_x,np.ravel(test_y))
        
        model = {'Name' : name,
                'Classes': ['cancer', 'no cancer'],
                'Date': '2020-10-01',
                'Datasets': ['madelon', 'musk'],
                'Training_Score' : score_ytrain,
                 'Test_Score' : score_ytest
                }
        
        #print("classifier %s, train score %.5f \n" %(name,score_ytrain))
        #print("classifier %s, test score %.5f \n" %(name,score_ytest))
        print("classifier %s " %(name), classification_report(np.ravel(test_y), pred_ytest, digits = 4))
        #print("probabilty of test", probs)
        model_return[name] = pred_ytrain
    return model_return

In [7]:
madelon_return = data_classification(train_x = Madelon_X, train_y = Madelon_Y, test_x = Madelon_x ,test_y = Madelon_y)

GaussianNB(priors=None, var_smoothing=1e-09)
classifier NB                precision    recall  f1-score   support

         0.0     0.4959    0.4667    0.4808       900
         1.0     0.4963    0.5256    0.5105       900

    accuracy                         0.4961      1800
   macro avg     0.4961    0.4961    0.4957      1800
weighted avg     0.4961    0.4961    0.4957      1800

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
classifier RT                precision    recall  f1-score   support

         0.0     0.5039    0.5033    0.5036       900
         1.0     0.5039    0.5044    0.5042       900

    accuracy                         0.5039      1800
   macro avg     0.5039

In [8]:
musk_return = data_classification(train_x = Musk_X, train_y = Musk_Y, test_x = Musk_x ,test_y = Musk_y)


GaussianNB(priors=None, var_smoothing=1e-09)
classifier NB                precision    recall  f1-score   support

         0.0     0.6724    0.8773    0.7613       269
         1.0     0.7360    0.4444    0.5542       207

    accuracy                         0.6891       476
   macro avg     0.7042    0.6609    0.6578       476
weighted avg     0.7000    0.6891    0.6712       476

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
classifier RT                precision    recall  f1-score   support

         0.0     0.7438    0.4424    0.5548       269
         1.0     0.5253    0.8019    0.6348       207

    accuracy                         0.5987       476
   macro avg     0.6345

## Proposed Algorithm for Feature Selection

In [9]:
import warnings
warnings.filterwarnings('ignore')

class feature_optimization:
    def __init__(self, trainX, trainY, testX, testY, population, obj_function):
        self.trainX = trainX
        self.trainY = trainY
        self.testX  = testX
        self.testY  = testY
        self.P = population
        if obj_function == 'ER':
            self.objective_function = fx_ER
            self.jaya_function = self.jaya_ER
        elif obj_function == 'AUC':
            self.objective_function = fx_AUC
            self.jaya_function = self.jaya_AUC
        else:
            print('please select the correct fx function-- ER [Error rate] or AUC [Area under the curve]')
        
        self.dsp_binary_data = generate_binary_dataset(self.P, self.objective_function)
        
        
    def __call__(self):
        self.X = self.dsp_binary_data(self.trainX, self.testX, self.trainY, self.testY)
        optimized_population = self.jaya_function(self.X)
        #print('OP_Features ', optimized_population.shape)
        trainOP, testOP = self.optimized_training_test_sets(optimized_population)
        return trainOP, testOP
        
    def __objective_function(self, X_new): 
        FV = []
        for n in range(len(X_new)):
            #print('xnew ', X_new.loc[n])
            idx ,  = np.where(X_new.loc[n]==1)
            
            temp_train = self.trainX[idx]
            temp_test = self.testX[idx]
            
           
            FV.append(self.objective_function(X_train = temp_train,
                          x_test  = temp_test,
                          Y_train = self.trainY,
                          y_test  = self.testY)
                     )

        return FV
    
    def optimized_training_test_sets(self, X):
        idx,  = np.where(X.loc[0]==1)
        #print("feature IDX ", idx)
        temp_train = self.trainX[idx]
        temp_test = self.testX[idx]
        
        return temp_train, temp_test
   
    

    def binary_x_new(self,X_new):
        '''
        PURPOSE: to convert x_new value into binary [0, 1] 
        INPUT : output of Jaya algorithm line 36
        OUTPUT: binary value of x_new
        '''
        #probability_jaya_x  = 1/(1 + np.exp(-2 * X_new) - 0.25)   # OUR PROPOSED ONE 
        probability_jaya_x  = 1/(1 + np.exp(-10 * X_new) - 0.5)    # PAPER FUNCTION
        #print('probability_jaya_x ', probability_jaya_x)
        random_r =  np.round(np.random.random(1),2)
        prob = []
        for p in probability_jaya_x:
            if p > random_r:
                prob.append(1.0)
            else:
                prob.append(0.0)
        return prob    

    def jaya_ER(self,x):
        '''
        INPUT:
        bextX is with the smaller fx value, 
        worstX, 
        r1 and r2 are two random numbers between 0 and 1 
        '''

        MaxIter = 100
        print('Processing .....')
        for s in range(MaxIter):      # line 6
            if len(x) >1:
                
                #print('=================    ITERATION ==============', s)
                X_new = pd.DataFrame()
                bestX = x['fx'].idxmin(axis=1)
                worstX = x['fx'].idxmax(axis=1)
                #print('BEST ', bestX, 'WORST ', worstX)
                #print('INITIAL f(x) BEST ', x['fx'].min())
                #print('INITIAL f(x) WORST ', x['fx'].max())
                for i in range(len(x.columns)-1):
                    r1 = np.round(np.random.random(1),2)
                    r2 = np.round(np.random.random(1),2)

                    jaya_X = (x[x.columns[i]] + r1 * (bestX - abs(x[x.columns[i]])) - r2*(worstX - abs(x[x.columns[i]])))

                    X_new[x.columns[i]] = self.binary_x_new(jaya_X)

                X_new['fx'] = self.__objective_function(X_new)
                #print(X_new)

                for d in range(len(x)):
                    remove_index = []
                    if(X_new['fx'].iloc[d] < x['fx'].iloc[d]):
                        x['fx'].iloc[d] = X_new['fx'].iloc[d]
                    
                    else:
                                              
                        remove_index.append(d)
                        
                
                x =x.drop(index=x.index[[remove_index]])
            else:
                print('Only one population left')
                break;
                #print(x)
        return x

    def jaya_AUC(self,x):
        '''
        INPUT:
        bextX is with the larger fx value, 
        worstX, 
        r1 and r2 are two random numbers between 0 and 1 
        '''

        MaxIter = 100
        print('Processing ........')
        for s in range(MaxIter):      # line 6
            # print("# of Populations ", len(x))
            if len(x) >1:
               
                #print('=================    ITERATION ==============', s)
                X_new = pd.DataFrame()
                bestX = x['fx'].idxmax(axis=1)
                worstX = x['fx'].idxmin(axis=1)
                #print('BEST ', bestX, 'WORST ', worstX)
                #print('INITIAL f(x) BEST ', x['fx'].max())
                #print('INITIAL f(x) WORST ', x['fx'].min())
                for i in range(len(x.columns)-1):
                    r1 = np.round(np.random.random(1),2)
                    r2 = np.round(np.random.random(1),2)

                    jaya_X = (x[x.columns[i]] + r1 * (bestX - abs(x[x.columns[i]])) - r2*(worstX - abs(x[x.columns[i]])))

                    X_new[x.columns[i]] = self.binary_x_new(jaya_X)

                X_new['fx'] = self.__objective_function(X_new)
                #print(X_new)

                for d in range(len(x)):
                    remove_index = []
                    if(X_new['fx'].iloc[d] > x['fx'].iloc[d]):

                        x['fx'].iloc[d] = X_new['fx'].iloc[d]
                    else:
                                              
                        remove_index.append(d)
                        
                
                x =x.drop(index=x.index[[remove_index]])
            else:
                print('Only one population left')
                break;
                
        return x 
    
def error_rate(pred, orig):
    '''
    PURPOSE: Calculate the rate of accuracy of a classifier. 
    INPUT: pred = predicted class by a classifier orig: original class. [type binary array]. 
    OUTPUT: float error rate. 
    
    '''
    error = np.mean((pred != orig).astype(float))
    return error


## NOTE: Covert fx_ER and fx_AUC into classes, so we can transfer the classifier as well. 
'''
class fx_function_selection:
    def __init__(self, model_selected, function_selected):
        
        
        self.function_selected = function
'''   
def fx_ER(X_train, x_test, Y_train, y_test):
    # We can choose any classifier here. 
    clf = GaussianNB()
    #clf = KNeighborsClassifier(n_neighbors=10)
    clf.fit(X_train, np.ravel(Y_train))
    pred_y = clf.predict(x_test)
    return error_rate(pred_y, y_test)

def fx_AUC(X_train, x_test, Y_train, y_test):
    # We can choose any classifier here.  
    clf = GaussianNB()
    #clf = KNeighborsClassifier(n_neighbors=10)
    clf.fit(X_train, np.ravel(Y_train))
    pred_y = clf.predict_proba(x_test)
    # keep probabilities for the positive outcome only
    pred_y  = pred_y [:, 1]
    lr_auc = roc_auc_score(y_test, pred_y )

    return lr_auc

class binary_sets:
    def __init__(self, D):
        self.D = D
    def __call__(self, population):
        idx = (population.keys())
        binary = np.zeros(self.D)
        np.put(binary, idx,1)
        return binary
    
        
class feature_subset:
    def __init__(self, list_of_features):
        self.features = list_of_features
        
        
    def __call__(self, dataframe):
        #columns = [f  for f in self.args]
                
        #print(self.features)  
        new_df = dataframe[self.features]
        return new_df 
    
class generate_binary_dataset:
    def __init__(self, population, fx):
        self.P = population
        self.f_x = fx
        
        return None
    def __call__(self,X_train, x_test, Y_train, y_test):
        D = len(X_train.columns)
        col_names = X_train.columns
        binary_dsp = binary_sets(D)
        min_features = int(D/3)
        
        X = {}
        FV = []
        
        
        for i in range(self.P):
            #print('=================================  Population: ', i, "===============================")
            random_D = np.arange(2,(np.random.randint(min_features, D)))
            #print(i, '# of features: ', len(random_D))
            feature_sub = feature_subset(random_D)
            
            tem_train_X = feature_sub(X_train)
            tem_test_X  = feature_sub(x_test)
            
            
                       
            FV.append(self.f_x(X_train = tem_train_X,
                          x_test  = tem_test_X,
                          Y_train = Y_train,
                          y_test  = y_test)
                     )
            
            
            X[i] =  binary_dsp(tem_test_X)
        Xx = pd.DataFrame(X.values(), columns = col_names ,index = X.keys())
        
        Xx['fx'] = FV
        
        #print(Xx.head(10))
        return Xx
    
    
    

## TESTING PROPOSED STRATEGY

In [10]:
for a in range(10):
    print('================================\n -----------RESULT # ------- ', a)
    dsp = feature_optimization(trainX = Musk_X,
                               trainY = Musk_Y,
                               testX = Musk_x,
                               testY = Musk_y, 
                               population = 10,
                               obj_function= 'ER')

    MuskX_OP, Muskx_OP = dsp()
    print('Number of Features ', MuskX_OP.shape[1])
    musk_return = data_classification(train_x = MuskX_OP, train_y = Musk_Y, 
                                  test_x = Muskx_OP ,test_y = Musk_y)



 -----------RESULT # -------  0
Processing .....
Only one population left
Number of Features  72
GaussianNB(priors=None, var_smoothing=1e-09)
classifier NB                precision    recall  f1-score   support

         0.0     0.7081    0.8476    0.7716       269
         1.0     0.7338    0.5459    0.6260       207

    accuracy                         0.7164       476
   macro avg     0.7209    0.6967    0.6988       476
weighted avg     0.7192    0.7164    0.7083       476

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
classifier RT                precision    recall  f1-score   support

         0.0     0.8271    0.8178    0.8224       269
         1.0     0.7667    0.7778  

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)
classifier LDA                precision    recall  f1-score   support

         0.0     0.7616    0.8550    0.8056       269
         1.0     0.7759    0.6522    0.7087       207

    accuracy                         0.7668       476
   macro avg     0.7687    0.7536    0.7571       476
weighted avg     0.7678    0.7668    0.7634       476

 -----------RESULT # -------  4
Processing .....
Only one population left
Number of Features  85
GaussianNB(priors=None, var_smoothing=1e-09)
classifier NB                precision    recall  f1-score   support

         0.0     0.7085    0.8401    0.7687       269
         1.0     0.7261    0.5507    0.6264       207

    accuracy                         0.7143       476
   macro avg     0.7173    0.6954    0.6975       476
weighted avg     0.7161    0.7143    0.7068       476

DecisionTreeRegressor

classifier KNN                precision    recall  f1-score   support

         0.0     0.8433    0.9405    0.8893       269
         1.0     0.9091    0.7729    0.8355       207

    accuracy                         0.8676       476
   macro avg     0.8762    0.8567    0.8624       476
weighted avg     0.8719    0.8676    0.8659       476

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)
classifier LDA                precision    recall  f1-score   support

         0.0     0.7191    0.7993    0.7570       269
         1.0     0.6949    0.5942    0.6406       207

    accuracy                         0.7101       476
   macro avg     0.7070    0.6967    0.6988       476
weighted avg     0.7086    0.7101    0.7064       476

 -----------RESULT # -------  8
Processing .....
Only one population left
Number of Features  122
GaussianNB(priors=None, var_smoothing=1e-09)
classifier NB      

In [11]:
for a in range(20):
    print('================================\n -----------RESULT # ------- ', a)
    dsp = feature_optimization(trainX = Madelon_X,
                               trainY = Madelon_Y,
                               testX = Madelon_x,
                               testY = Madelon_y, 
                               population = 30,
                               obj_function= 'ER')

    MadelonX_OP, Madelonx_OP = dsp()
    print('Number of Features ', MadelonX_OP.shape[1])
    Madelon_return = data_classification(train_x = MadelonX_OP, train_y = Madelon_Y, 
                                         test_x = Madelonx_OP ,test_y = Madelon_y)

 -----------RESULT # -------  0
Processing .....
Only one population left
Number of Features  417
GaussianNB(priors=None, var_smoothing=1e-09)
classifier NB                precision    recall  f1-score   support

         0.0     0.5035    0.4778    0.4903       900
         1.0     0.5032    0.5289    0.5157       900

    accuracy                         0.5033      1800
   macro avg     0.5033    0.5033    0.5030      1800
weighted avg     0.5033    0.5033    0.5030      1800

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
classifier RT                precision    recall  f1-score   support

         0.0     0.4951    0.5022    0.4986       900
         1.0     0.4949    0.4878 

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)
classifier LDA                precision    recall  f1-score   support

         0.0     0.5017    0.4867    0.4941       900
         1.0     0.5016    0.5167    0.5090       900

    accuracy                         0.5017      1800
   macro avg     0.5017    0.5017    0.5016      1800
weighted avg     0.5017    0.5017    0.5016      1800

 -----------RESULT # -------  4
Processing .....
Only one population left
Number of Features  235
GaussianNB(priors=None, var_smoothing=1e-09)
classifier NB                precision    recall  f1-score   support

         0.0     0.5000    0.4778    0.4886       900
         1.0     0.5000    0.5222    0.5109       900

    accuracy                         0.5000      1800
   macro avg     0.5000    0.5000    0.4998      1800
weighted avg     0.5000    0.5000    0.4998      1800

DecisionTreeRegresso

classifier KNN                precision    recall  f1-score   support

         0.0     0.4897    0.6056    0.5415       900
         1.0     0.4833    0.3689    0.4184       900

    accuracy                         0.4872      1800
   macro avg     0.4865    0.4872    0.4799      1800
weighted avg     0.4865    0.4872    0.4799      1800

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)
classifier LDA                precision    recall  f1-score   support

         0.0     0.4994    0.4756    0.4872       900
         1.0     0.4995    0.5233    0.5111       900

    accuracy                         0.4994      1800
   macro avg     0.4994    0.4994    0.4992      1800
weighted avg     0.4994    0.4994    0.4992      1800

 -----------RESULT # -------  8
Processing .....
Only one population left
Number of Features  255
GaussianNB(priors=None, var_smoothing=1e-09)
classifier NB      

classifier KNN                precision    recall  f1-score   support

         0.0     0.4863    0.6100    0.5412       900
         1.0     0.4769    0.3556    0.4074       900

    accuracy                         0.4828      1800
   macro avg     0.4816    0.4828    0.4743      1800
weighted avg     0.4816    0.4828    0.4743      1800

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)
classifier LDA                precision    recall  f1-score   support

         0.0     0.5092    0.4911    0.5000       900
         1.0     0.5086    0.5267    0.5175       900

    accuracy                         0.5089      1800
   macro avg     0.5089    0.5089    0.5087      1800
weighted avg     0.5089    0.5089    0.5087      1800

 -----------RESULT # -------  12
Processing .....
Only one population left
Number of Features  406
GaussianNB(priors=None, var_smoothing=1e-09)
classifier NB     

classifier KNN                precision    recall  f1-score   support

         0.0     0.4898    0.6133    0.5446       900
         1.0     0.4829    0.3611    0.4132       900

    accuracy                         0.4872      1800
   macro avg     0.4864    0.4872    0.4789      1800
weighted avg     0.4864    0.4872    0.4789      1800

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)
classifier LDA                precision    recall  f1-score   support

         0.0     0.5046    0.4867    0.4955       900
         1.0     0.5043    0.5222    0.5131       900

    accuracy                         0.5044      1800
   macro avg     0.5045    0.5044    0.5043      1800
weighted avg     0.5045    0.5044    0.5043      1800

 -----------RESULT # -------  16
Processing .....
Only one population left
Number of Features  269
GaussianNB(priors=None, var_smoothing=1e-09)
classifier NB     

classifier KNN                precision    recall  f1-score   support

         0.0     0.5090    0.5644    0.5353       900
         1.0     0.5112    0.4556    0.4818       900

    accuracy                         0.5100      1800
   macro avg     0.5101    0.5100    0.5085      1800
weighted avg     0.5101    0.5100    0.5085      1800

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)
classifier LDA                precision    recall  f1-score   support

         0.0     0.5017    0.4800    0.4906       900
         1.0     0.5016    0.5233    0.5122       900

    accuracy                         0.5017      1800
   macro avg     0.5017    0.5017    0.5014      1800
weighted avg     0.5017    0.5017    0.5014      1800

